In [3]:
## Diabetic Retinopathy

In [9]:
### Preprocessing

In [24]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Define input and output directories
input_dir = "dataset/"  # Update with your actual dataset path
output_dir_left = "processed/left/"
output_dir_right = "processed/right/"

# Ensure output directories exist
os.makedirs(output_dir_left, exist_ok=True)
os.makedirs(output_dir_right, exist_ok=True)

# Check if input directory exists
if not os.path.exists(input_dir):
    raise FileNotFoundError(f"❌ Dataset directory '{input_dir}' not found. Please check the path.")

def preprocess_and_save(image_path, save_path):
    """Load, resize, normalize, and save the image."""
    img = cv2.imread(image_path)  # Load image
    img = cv2.resize(img, (256, 256))  # Resize to 256x256
    img = img / 127.5 - 1  # Normalize pixel values to (-1,1)
    np.save(save_path, img)  # Save as .npy for efficient loading

# Process images
for filename in tqdm(os.listdir(input_dir)):
    if filename.endswith(".jpg"):
        img_path = os.path.join(input_dir, filename)
        
        if "_L.jpg" in filename:
            save_path = os.path.join(output_dir_left, filename.replace(".jpg", ".npy"))
            preprocess_and_save(img_path, save_path)
        elif "_R.jpg" in filename:
            save_path = os.path.join(output_dir_right, filename.replace(".jpg", ".npy"))
            preprocess_and_save(img_path, save_path)

print("✅ Preprocessing complete! Images saved as .npy for efficient training.")

100%|██████████████████████████████████████████████████████████████████| 3927/3927 [00:10<00:00, 391.58it/s]

✅ Preprocessing complete! Images saved as .npy for efficient training.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import time

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define hyperparameters
batch_size = 64
epochs = 1000
lr = 0.0002
latent_dim = 100
image_size = 256

# Define dataset directories
data_dir_left = "processed/left/"
data_dir_right = "processed/right/"

# Custom dataset loader
class RetinalDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.files = [f for f in os.listdir(root_dir) if f.endswith(".npy")]
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.files[idx])
        img = np.load(img_path)
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)  # Convert to tensor, rearrange channels
        return img.to(device), 0  # Dummy label

# Load datasets
dataset_left = RetinalDataset(data_dir_left)
dataset_right = RetinalDataset(data_dir_right)

dataloader_left = DataLoader(dataset_left, batch_size=batch_size, shuffle=True)
dataloader_right = DataLoader(dataset_right, batch_size=batch_size, shuffle=True)

# Define Generator
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, image_size * image_size * 3),
            nn.Tanh()
        )
    
    def forward(self, z):
        img = self.model(z)
        img = img.view(-1, 3, image_size, image_size)
        return img

# Define Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(image_size * image_size * 3, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# Training function
def train_gan(dataloader, eye):
    print(f"\n🔵 Training GAN for {eye} eye images on {device}...")
    generator = Generator(latent_dim).to(device)
    discriminator = Discriminator().to(device)
    adversarial_loss = nn.BCELoss().to(device)
    optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    start_time = time.time()
    
    for epoch in range(epochs):
        epoch_start = time.time()
        
        for i, (imgs, _) in enumerate(dataloader):
            batch_size = imgs.size(0)
            real_imgs = imgs.to(device)
            
            # Adversarial ground truths
            valid = torch.ones(batch_size, 1, device=device)
            fake = torch.zeros(batch_size, 1, device=device)
            
            # Train Generator
            optimizer_G.zero_grad()
            z = torch.randn(batch_size, latent_dim, device=device)
            gen_imgs = generator(z)
            g_loss = adversarial_loss(discriminator(gen_imgs), valid)
            g_loss.backward()
            optimizer_G.step()
            
            # Train Discriminator
            optimizer_D.zero_grad()
            real_loss = adversarial_loss(discriminator(real_imgs), valid)
            fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
            d_loss = (real_loss + fake_loss) / 2
            d_loss.backward()
            optimizer_D.step()
        
        epoch_time = time.time() - epoch_start
        print(f"Epoch {epoch+1}/{epochs} | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f} | Time: {epoch_time:.2f} sec")
    
    total_time = time.time() - start_time
    print(f"✅ {eye.capitalize()} Eye GAN Training Complete! Total Training Time: {total_time:.2f} sec")
    torch.save(generator.state_dict(), f"generator_{eye}_1000epoch.pth")
    torch.save(discriminator.state_dict(), f"discriminator_{eye}_1000epoch.pth")
    
    print(f"📝 Model saved: generator_{eye}_1000epoch.pth and discriminator_{eye}_1000epoch.pth")

# Train separate GANs for left and right eyes
train_gan(dataloader_left, "left")

print("🎯 Training Finished! All models saved.")


Using device: cuda

🔵 Training GAN for left eye images on cuda...
Epoch 1/1000 | D Loss: 0.5627 | G Loss: 0.4007 | Time: 4.70 sec
Epoch 2/1000 | D Loss: 0.7222 | G Loss: 0.2703 | Time: 2.43 sec
Epoch 3/1000 | D Loss: 0.7468 | G Loss: 0.2550 | Time: 2.21 sec
Epoch 4/1000 | D Loss: 1.2004 | G Loss: 0.0961 | Time: 2.33 sec
Epoch 5/1000 | D Loss: 0.6404 | G Loss: 0.4188 | Time: 2.13 sec
Epoch 6/1000 | D Loss: 0.3527 | G Loss: 0.7515 | Time: 2.11 sec
Epoch 7/1000 | D Loss: 0.6029 | G Loss: 0.3620 | Time: 2.36 sec
Epoch 8/1000 | D Loss: 0.3738 | G Loss: 0.8113 | Time: 2.19 sec
Epoch 9/1000 | D Loss: 0.3572 | G Loss: 0.7750 | Time: 2.42 sec
Epoch 10/1000 | D Loss: 0.2961 | G Loss: 0.8595 | Time: 2.57 sec
Epoch 11/1000 | D Loss: 0.3045 | G Loss: 0.8740 | Time: 3.85 sec
Epoch 12/1000 | D Loss: 0.4047 | G Loss: 1.0551 | Time: 8.89 sec
Epoch 13/1000 | D Loss: 0.3995 | G Loss: 1.0987 | Time: 9.08 sec
Epoch 14/1000 | D Loss: 0.4652 | G Loss: 0.6857 | Time: 10.55 sec
Epoch 15/1000 | D Loss: 0.4183 |